In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

**Read data**

In [2]:
X = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")

**target and predictors**

In [6]:
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

## 1. Drop columns with missing values

In [7]:
cols_missing = [col for col in X.columns if X[col].isnull().any()]
X.drop(cols_missing, axis=1, inplace=True)
X_test.drop(cols_missing, axis=1, inplace=True)

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

**score_dataset**

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [20]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Approach 1: Drop columns with categorical data

In [26]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

In [27]:
print('MAE: ', end="")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE: 17952.591404109586


### Approach 2: Ordinal Encoding

_part A_

In [ ]:
# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

**categorical columns**

In [29]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

In [32]:
good_cols = [col for col in object_cols if set(X_valid[col]).issubset(X_train[col])]

bad_cols = list(set(object_cols) - set(good_cols))

In [34]:
print("Good columns:", good_cols)
print()
print("Bad columns:", bad_cols)

Good columns: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']

Bad columns: ['Functional', 'RoofMatl', 'Condition2']


_part B_

In [36]:
from sklearn.preprocessing import OrdinalEncoder

**drop bad columns**

In [38]:
label_X_train = X_train.drop(bad_cols, axis=1)
label_X_valid = X_valid.drop(bad_cols, axis=1)

**label encoder**

In [39]:
OE = OrdinalEncoder()
label_X_train[good_cols] = OE.fit_transform(label_X_train[good_cols])
label_X_valid[good_cols] = OE.fit_transform(label_X_valid[good_cols])

In [40]:
print('MAE: ', end="")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE: 17515.441506849314


### Approach 3: Nominal Encoding

**unique entries in each categorical columns**

In [43]:
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [44]:
high_ununique_cols = [k for k in d.keys() if d[k] > 10]
high_ununique_cols

['Neighborhood', 'Exterior1st', 'Exterior2nd']

In [46]:
low_nunique = [col for col in object_cols if X_train[col].nunique() < 10]

high_nunique = list(set(object_cols) - set(low_nunique))

In [47]:
print(low_nunique)
print(high_nunique)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']
['Neighborhood', 'Exterior1st', 'Exterior2nd']


**OneHot Encoder**

In [48]:
from sklearn.preprocessing import OneHotEncoder

In [52]:
OHE = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OHE.fit_transform(X_train[low_nunique]))
OH_cols_valid = pd.DataFrame(OHE.transform(X_valid[low_nunique]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([OH_cols_train, num_X_train], axis=1)
OH_X_valid = pd.concat([OH_cols_valid, num_X_valid], axis=1)

In [123]:
print('MAE: ', end="")
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE: 17643.61551369863


# Test

In [66]:
from sklearn.impute import SimpleImputer

In [71]:
my1 = SimpleImputer(strategy='mean')
my2 = SimpleImputer(strategy='most_frequent')

In [70]:
obj_cols = [col for col in X_test if X_test[col].dtype=='object']
num_cols = list(set(X_test.columns) - set(obj_cols))

print(num_cols)
print()
print(obj_cols)

['KitchenAbvGr', 'FullBath', 'MoSold', 'OverallCond', 'YearRemodAdd', 'LowQualFinSF', 'TotRmsAbvGrd', 'PoolArea', '3SsnPorch', 'Id', '2ndFlrSF', 'Fireplaces', 'ScreenPorch', 'BedroomAbvGr', 'OverallQual', 'YearBuilt', 'LotArea', 'YrSold', 'BsmtUnfSF', 'BsmtFinSF2', 'BsmtFullBath', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', '1stFlrSF', 'BsmtFinSF1', 'BsmtHalfBath', 'HalfBath', 'TotalBsmtSF', 'MiscVal', 'MSSubClass', 'GarageCars', 'EnclosedPorch', 'GrLivArea']

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']


In [74]:
num_X_test = pd.DataFrame(my1.fit_transform(X_test[num_cols]))
obj_X_test = pd.DataFrame(my2.fit_transform(X_test[obj_cols]))

num_X_test.columns = X_test[num_cols].columns
obj_X_test.columns = X_test[obj_cols].columns

pre_X_test = pd.concat([num_X_test, obj_X_test], axis=1)

In [75]:
pre_X_test.head()

,KitchenAbvGr,FullBath,MoSold,OverallCond,YearRemodAdd,LowQualFinSF,TotRmsAbvGrd,PoolArea,3SsnPorch,Id,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,1.0,1.0,6.0,6.0,1961.0,0.0,5.0,0.0,0.0,1461.0,...,TA,CBlock,GasA,TA,Y,TA,Typ,Y,WD,Normal
1,1.0,1.0,6.0,6.0,1958.0,0.0,6.0,0.0,0.0,1462.0,...,TA,CBlock,GasA,TA,Y,Gd,Typ,Y,WD,Normal
2,1.0,2.0,3.0,5.0,1998.0,0.0,6.0,0.0,0.0,1463.0,...,TA,PConc,GasA,Gd,Y,TA,Typ,Y,WD,Normal
3,1.0,2.0,6.0,6.0,1998.0,0.0,7.0,0.0,0.0,1464.0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
4,1.0,2.0,1.0,5.0,1992.0,0.0,5.0,0.0,0.0,1465.0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal


In [79]:
pre_X_test.isnull().sum()

KitchenAbvGr     0
FullBath         0
MoSold           0
OverallCond      0
YearRemodAdd     0
                ..
KitchenQual      0
Functional       0
PavedDrive       0
SaleType         0
SaleCondition    0
Length: 61, dtype: int64

In [109]:
pre_X_test_1 = pd.DataFrame(OHE.transform(pre_X_test[low_nunique]))

In [110]:
pre_X_test_1.index = pre_X_test.index

In [111]:
pre_X_test_1.head()

,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [113]:
num_pre_X_test = pre_X_test.drop(object_cols, axis=1)

In [114]:
num_pre_X_test.head()

,KitchenAbvGr,FullBath,MoSold,OverallCond,YearRemodAdd,LowQualFinSF,TotRmsAbvGrd,PoolArea,3SsnPorch,Id,...,1stFlrSF,BsmtFinSF1,BsmtHalfBath,HalfBath,TotalBsmtSF,MiscVal,MSSubClass,GarageCars,EnclosedPorch,GrLivArea
0,1.0,1.0,6.0,6.0,1961.0,0.0,5.0,0.0,0.0,1461.0,...,896.0,468.0,0.0,0.0,882.0,0.0,20.0,1.0,0.0,896.0
1,1.0,1.0,6.0,6.0,1958.0,0.0,6.0,0.0,0.0,1462.0,...,1329.0,923.0,0.0,1.0,1329.0,12500.0,20.0,1.0,0.0,1329.0
2,1.0,2.0,3.0,5.0,1998.0,0.0,6.0,0.0,0.0,1463.0,...,928.0,791.0,0.0,1.0,928.0,0.0,60.0,2.0,0.0,1629.0
3,1.0,2.0,6.0,6.0,1998.0,0.0,7.0,0.0,0.0,1464.0,...,926.0,602.0,0.0,1.0,926.0,0.0,60.0,2.0,0.0,1604.0
4,1.0,2.0,1.0,5.0,1992.0,0.0,5.0,0.0,0.0,1465.0,...,1280.0,263.0,0.0,0.0,1280.0,0.0,120.0,2.0,0.0,1280.0


In [117]:
final_X_test = pd.concat([pre_X_test_1, num_pre_X_test], axis=1)

In [119]:
final_X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,1stFlrSF,BsmtFinSF1,BsmtHalfBath,HalfBath,TotalBsmtSF,MiscVal,MSSubClass,GarageCars,EnclosedPorch,GrLivArea
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,896.0,468.0,0.0,0.0,882.0,0.0,20.0,1.0,0.0,896.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1329.0,923.0,0.0,1.0,1329.0,12500.0,20.0,1.0,0.0,1329.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,928.0,791.0,0.0,1.0,928.0,0.0,60.0,2.0,0.0,1629.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,926.0,602.0,0.0,1.0,926.0,0.0,60.0,2.0,0.0,1604.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1280.0,263.0,0.0,0.0,1280.0,0.0,120.0,2.0,0.0,1280.0


In [127]:
final_X_test.shape

(1459, 156)

In [124]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(OH_X_train, y_train)
preds_test = model.predict(final_X_test)

In [131]:
X_test_full = pd.read_csv('test.csv', index_col='Id')

In [133]:
output = pd.DataFrame({'Id': X_test_full.index, 'SalePrice': preds_test})
output.to_csv('submission_3.csv', index=False)

In [132]:
X_test_full.index

Int64Index([1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470,
            ...
            2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919],
           dtype='int64', name='Id', length=1459)